In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from mlxtend.frequent_patterns import apriori, association_rules
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/uci-online-retail-ii-data-set/online_retail_II.xlsx


In [2]:
data = pd.read_excel("../input/uci-online-retail-ii-data-set/online_retail_II.xlsx", sheet_name = "Year 2010-2011")

In [3]:
data.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
# iadeleri sildik
"""
iadeler = []
for i,j in enumerate(df["Invoice"].values):
    if str(j).startswith("C"):
        iadeler.append(i)
    

df.drop(iadeler, inplace=True) 
"""
data=data[~data["Invoice"].astype(str).str.startswith("C")]

In [5]:
data.isna().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    134697
Country             0
dtype: int64

In [6]:
ba = data.copy()

In [7]:
ba.dropna(subset = ['Description'], inplace=True) # nanları attık.

In [8]:
ba.isna().sum()

Invoice             0
StockCode           0
Description         0
Quantity            0
InvoiceDate         0
Price               0
Customer ID    133243
Country             0
dtype: int64

In [9]:
ba.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [10]:
ba.shape

(531168, 8)

In [11]:
ba["Description"] = ba["Description"].astype(str).apply(lambda x: x.strip())

In [12]:
ba=ba[~ba["Description"].astype(str).str.startswith("wrong")]

In [13]:
ba.shape

(531153, 8)

In [14]:
# işimize yaramayan column'ları attık.
ba.drop(columns=['StockCode', "InvoiceDate", "Price", "Country", "Customer ID"], inplace=True)

In [15]:
ba['Description'].tail()

541905        CHILDREN'S APRON DOLLY GIRL
541906       CHILDRENS CUTLERY DOLLY GIRL
541907    CHILDRENS CUTLERY CIRCUS PARADE
541908       BAKING SET 9 PIECE RETROSPOT
541909                            POSTAGE
Name: Description, dtype: object

In [16]:
ba.head()

,Invoice,Description,Quantity
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6
1,536365,WHITE METAL LANTERN,6
2,536365,CREAM CUPID HEARTS COAT HANGER,8
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6


In [17]:
ba.groupby(['Invoice','Description'])['Description'].count()

Invoice  Description                        
536365   CREAM CUPID HEARTS COAT HANGER         1
         GLASS STAR FROSTED T-LIGHT HOLDER      1
         KNITTED UNION FLAG HOT WATER BOTTLE    1
         RED WOOLLY HOTTIE WHITE HEART.         1
         SET 7 BABUSHKA NESTING BOXES           1
                                               ..
581587   POSTAGE                                1
         SPACEBOY LUNCH BOX                     1
A563185  Adjust bad debt                        1
A563186  Adjust bad debt                        1
A563187  Adjust bad debt                        1
Name: Description, Length: 520325, dtype: int64

In [18]:
# Invoice ve Description'a göre gruplayıp Quantityye göre toplayıp unstack yapıyor
branch_order = (ba
          .groupby(['Invoice', 'Description'])['Quantity'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('Invoice'))

In [19]:
branch_order.head()

Description,*Boombox Ipod Classic,*USB Office Mirror Ball,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,...,water damage,water damaged,website fixed,wet,wet boxes,wet damaged,wet pallet,wet rusty,wet/rusty,wet?
Invoice,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
encoded = branch_order.applymap(lambda x: 1 if x != 0 else 0) # 

In [21]:
encoded.head()

Description,*Boombox Ipod Classic,*USB Office Mirror Ball,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,...,water damage,water damaged,website fixed,wet,wet boxes,wet damaged,wet pallet,wet rusty,wet/rusty,wet?
Invoice,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
freq_items = apriori(encoded, min_support=0.04, use_colnames=True, verbose=True)

Processing 3906 combinations | Sampling itemset size 2


In [23]:
freq_items

,support,itemsets
0,0.046370,(6 RIBBONS RUSTIC CHARM)
1,0.040204,(60 TEATIME FAIRY CAKE CASES)
2,0.047584,(ALARM CLOCK BAKELIKE GREEN)
3,0.051032,(ALARM CLOCK BAKELIKE RED)
4,0.070648,(ASSORTED COLOUR BIRD ORNAMENT)
...,...,...
59,0.109735,(WHITE HANGING HEART T-LIGHT HOLDER)
60,0.047147,(WOODEN FRAME ANTIQUE WHITE)
61,0.053411,(WOODEN PICTURE FRAME WHITE FINISH)
62,0.040544,(WOODLAND CHARLOTTE BAG)


In [24]:
freq_items.sort_values('support', ascending=False)

,support,itemsets
59,0.109735,(WHITE HANGING HEART T-LIGHT HOLDER)
20,0.101578,(JUMBO BAG RED RETROSPOT)
45,0.096577,(REGENCY CAKESTAND 3 TIER)
38,0.081865,(PARTY BUNTING)
30,0.075941,(LUNCH BAG RED RETROSPOT)
...,...,...
62,0.040544,(WOODLAND CHARLOTTE BAG)
52,0.040495,(SET OF 6 SPICE TINS PANTRY DESIGN)
1,0.040204,(60 TEATIME FAIRY CAKE CASES)
21,0.040058,(JUMBO BAG STRAWBERRY)


In [25]:
association_rules(freq_items, metric = 'confidence', min_threshold=0.4).sort_values(['support','confidence'], ascending=[False,False])


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(JUMBO BAG PINK POLKADOT),(JUMBO BAG RED RETROSPOT),0.059141,0.101578,0.040058,0.67734,6.668172,0.034051,2.784422


* JUMBO BAG PINK POLKADOT ve JUMBO BAG RED RETROSPOT tüm alışverişlerin %4'ünde birlikte bulunuyor. (support)
* JUMBO BAG PINK POLKADOT alanların %67'si JUMBO BAG RED RETROSPOT de alıyor. (confidence)
* JUMBO BAG PINK POLKADOT alımı JUMBO BAG RED RETROSPOT alımını 6.6 kat artırıyor. (lift)

# RFM

In [26]:
rfm = data.copy()

In [27]:
rfm.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [28]:
rfm.dropna(inplace=True)

In [29]:
rfm.drop(columns=['StockCode', 'Country'], inplace=True) # işimize yaramayan columnları attık.

In [30]:
rfm["Customer ID"] = rfm["Customer ID"].astype(int) # CustomerID'yi integer'a çevirdik. Çünkü çirkin duruyordu.

In [31]:
rfm.head()

,Invoice,Description,Quantity,InvoiceDate,Price,Customer ID
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850
1,536365,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850
2,536365,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850


In [32]:
# Fatura başına ortalama ne kadar kazanılmıştır? 
rfm['Total'] = rfm["Quantity"] * rfm['Price']

In [33]:
rfm.head()

,Invoice,Description,Quantity,InvoiceDate,Price,Customer ID,Total
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,15.30
1,536365,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,20.34
2,536365,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,22.00
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,20.34
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,20.34


In [34]:
# Fatura başı toplam kazanç
rfm.groupby('Invoice').agg({'Total':'sum'}) 

,Total
Invoice,
536365,139.12
536366,22.20
536367,278.73
536368,70.05
536369,17.85
...,...
581583,124.60
581584,140.64
581585,329.05


In [35]:
# Aykırı değerler var mı? Varsa kaç tane
for feature in ["Quantity","Price","Total"]:

    Q1 = rfm[feature].quantile(0.01)
    Q3 = rfm[feature].quantile(0.99)
    IQR = Q3-Q1
    upper = Q3 + 1.5*IQR
    lower = Q1 - 1.5*IQR

    if rfm[(rfm[feature] > upper) | (rfm[feature] < lower)].any(axis=None):
        print(feature,"yes")
        print(rfm[(rfm[feature] > upper) | (rfm[feature] < lower)].shape[0])
    else:
        print(feature, "no")

Quantity yes
963
Price yes
661
Total yes
903


### Recency

Recency (yenilik): Müşterinin son satın almasından bugüne kadar geçen süre

-- Bugünün tarihi - Son satın alma


In [36]:
rfm['InvoiceDate'].min() # ilk tarih

Timestamp('2010-12-01 08:26:00')

In [37]:
rfm['InvoiceDate'].max() # son tarih

Timestamp('2011-12-09 12:50:00')

In [38]:
today_date = dt.datetime(2011, 12 ,9) #bugünün tarihi

In [39]:
# Müşteriler en son ne zaman alışveriş yaptı?
rfm.groupby("Customer ID").agg({"InvoiceDate":"max"}).head()

,InvoiceDate
Customer ID,
12346,2011-01-18 10:01:00
12347,2011-12-07 15:52:00
12348,2011-09-25 13:13:00
12349,2011-11-21 09:51:00
12350,2011-02-02 16:01:00


In [40]:
# Bugünden itibaren kaç gün önce alışveriş yapıldı?
(today_date - rfm.groupby("Customer ID").agg({"InvoiceDate":"max"})).head() 

,InvoiceDate
Customer ID,
12346,324 days 13:59:00
12347,1 days 08:08:00
12348,74 days 10:47:00
12349,17 days 14:09:00
12350,309 days 07:59:00


In [41]:
temp_df = (today_date - rfm.groupby("Customer ID").agg({"InvoiceDate":"max"}))
temp_df.rename(columns={"InvoiceDate": "Recency"}, inplace = True)

In [42]:
temp_df.head()

,Recency
Customer ID,
12346,324 days 13:59:00
12347,1 days 08:08:00
12348,74 days 10:47:00
12349,17 days 14:09:00
12350,309 days 07:59:00


In [43]:
recency_df = temp_df["Recency"].apply(lambda x: x.days) # Günleri aldık.

In [44]:
recency_df.head()

Customer ID
12346    324
12347      1
12348     74
12349     17
12350    309
Name: Recency, dtype: int64

### Frequency

Frequency (Sıklık): Toplam satın alma sayısı.

In [45]:
temp_df = rfm.groupby(["Customer ID","Invoice"]).agg({"Invoice":"count"})

In [46]:
temp_df.head()

Invoice
Customer ID Invoice         
12346       541431         1
12347       537626        31
            542237        29
            549222        24
            556201        18

In [47]:
# Her müşterinin kaç faturası var?
temp_df.groupby("Customer ID").agg({"Invoice":"count"}).head()

,Invoice
Customer ID,
12346,1
12347,7
12348,4
12349,1
12350,1


In [48]:
freq_df = temp_df.groupby("Customer ID").agg({"Invoice":"sum"})
freq_df.rename(columns={"Invoice": "Frequency"}, inplace = True)
freq_df.head()

,Frequency
Customer ID,
12346,1
12347,182
12348,31
12349,73
12350,17


### Monetary

Monetary (Parasal Değer): Müşterinin yaptığı toplam harcama.

In [49]:
monetary_df = rfm.groupby("Customer ID").agg({"Total":"sum"})

In [50]:
monetary_df.head()

,Total
Customer ID,
12346,77183.60
12347,4310.00
12348,1797.24
12349,1757.55
12350,334.40


In [51]:
monetary_df.rename(columns={"Total": "Monetary"}, inplace = True)

In [52]:
print(recency_df.shape,freq_df.shape,monetary_df.shape)

(4339,) (4339, 1) (4339, 1)


In [53]:
# rfm adında yeni bir DataFrame oluşturup recency, frequency ve monetary'yi birleştirdik.
rfm = pd.concat([recency_df, freq_df, monetary_df],  axis=1) 

In [54]:
rfm.head()

,Recency,Frequency,Monetary
Customer ID,,,
12346,324,1,77183.60
12347,1,182,4310.00
12348,74,31,1797.24
12349,17,73,1757.55
12350,309,17,334.40


In [55]:
# Recency : En yakın tarihten en uzak tarihe göre 5'ten 1'e skorladık.
# Frequency : Sıklığa göre 1'den 5'e göre skorladık
# Monetary : Müşteriden kazanılan toplam paraya göre skorladık.
rfm["RecencyScore"] = pd.qcut(rfm['Recency'], 5, labels = [5, 4, 3, 2, 1])
rfm["FrequencyScore"] = pd.qcut(rfm['Frequency'], 5, labels = [1, 2, 3, 4, 5])
rfm["MonetaryScore"] = pd.qcut(rfm['Monetary'], 5, labels = [1, 2, 3, 4, 5])

In [56]:
rfm.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore
Customer ID,,,,,,
12346,324,1,77183.60,1,1,5
12347,1,182,4310.00,5,5,5
12348,74,31,1797.24,2,3,4
12349,17,73,1757.55,4,4,4
12350,309,17,334.40,1,2,2


In [57]:
(rfm['RecencyScore'].astype(str) + 
 rfm['FrequencyScore'].astype(str) + 
 rfm['MonetaryScore'].astype(str)).head()

Customer ID
12346    115
12347    555
12348    234
12349    444
12350    122
dtype: object

In [58]:
# Müşteri segmentlerini belirledik/tanımladık. 
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At Risk',
    r'[1-2]5': 'Can\'t Loose',
    r'3[1-2]': 'About to Sleep',
    r'33': 'Need Attention',
    r'[3-4][4-5]': 'Loyal Customers',
    r'41': 'Promising',
    r'51': 'New Customers',
    r'[4-5][2-3]': 'Potential Loyalists',
    r'5[4-5]': 'Champions'
}

In [59]:
# Regex'e göre Her müşteriyi segmentlere ayırdık.
rfm['Segment'] = rfm['RecencyScore'].astype(str) + rfm['FrequencyScore'].astype(str)
rfm['Segment'] = rfm['Segment'].replace(seg_map, regex=True)
rfm.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,Segment
Customer ID,,,,,,,
12346,324,1,77183.60,1,1,5,Hibernating
12347,1,182,4310.00,5,5,5,Champions
12348,74,31,1797.24,2,3,4,At Risk
12349,17,73,1757.55,4,4,4,Loyal Customers
12350,309,17,334.40,1,2,2,Hibernating


In [60]:
rfm[["Segment", "Recency","Frequency","Monetary"]].groupby("Segment").agg(["mean","count"])

Recency         Frequency           Monetary      
                           mean count        mean count         mean count
Segment                                                                   
About to Sleep        51.192547   322   15.987578   322   450.997174   322
At Risk              164.435852   569   56.859402   569   996.916872   569
Can't Loose          142.226190    84  181.666667    84  2370.705012    84
Champions              4.400990   606  289.031353   606  6960.915446   606
Hibernating          208.251397  1074   13.589385  1074   536.535672  1074
Loyal Customers       32.206854   817  157.116279   817  2845.732852   817
Need Attention        50.985366   205   41.736585   205   856.195854   205
New Customers          5.421053    57    7.578947    57  3618.697018    57
Potential Loyalists   14.668699   492   34.945122   492   915.486994   492
Promising             21.362832   113    7.707965   113   425.633628   113

In [61]:
rfm[rfm["Segment"] == "Need Attention"].head()


,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,Segment
Customer ID,,,,,,,
12372,70,52,1298.04,3,3,4,Need Attention
12413,65,38,758.10,3,3,3,Need Attention
12446,56,58,1002.31,3,3,4,Need Attention
12458,70,38,947.40,3,3,4,Need Attention
12475,52,36,723.38,3,3,3,Need Attention


In [62]:
need_att = pd.DataFrame()
need_att['Need Attention Customer ID'] = rfm[rfm['Segment'] == 'Need Attention'].index

In [63]:
need_att.to_csv('need_att.csv') # csv'ye çevirme